<a href="https://colab.research.google.com/github/elenadanchenkomn/data_analysis/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%92%D1%82%D0%BE%D1%80%D0%BE%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [ ]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2025-11-18 07:25:33--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2025-11-18 07:25:33--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json.1’

creds.json.1        100%[===================>]   2.30K  --.-KB/s    in 0s      

2025-11-18 07:25:34 (34.0 MB/s) - ‘creds.json.1’ save

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Небольшое описание столбцов в таблицах:  
**Лист1:** `student_id` - айди студента; `student_name` - имя студента; `installment` (Y/N) - факт наличия рассрочки у студента (Y - рассрочка есть, N - рассрочки нет).  
**Лист2:** `student_id` - айди студента; `last_payment_date` - дата последней полученной оплаты; `expected_payment_date` - дата, в которую ожидаем следующий платеж (рассчитывается от даты последнего платежа).  
**Лист3:** `student_id` - айди студента; `already_payed_amount` - уже выплаченная часть рассрочки; `left_to_pay` - оставшаяся (невыплаченная) часть; `one-time_payment` - единоразовый платеж; `installment_amount` - общая сумма, которая бралась в рассрочку.

Как примерно должен выглядеть результат:

In [ ]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [ ]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев** (183 дня).

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023). При расчетах отталкивайтесь от даты ожидаемого платежа, разницу между платежами берите 183 дня.

**Решение**

Напишите свое решение ниже

In [76]:
from datetime import datetime, date, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# Removed relativedelta as timedelta(days=183) is more appropriate based on problem description

# Указываем необходимые права доступа к таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

# Загружаем ключи аутентификации из файла json
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

# Авторизуемся в Google Sheets API
client = gspread.authorize(creds)
sheet1 = client.open_by_url('https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?gid=0#gid=0').worksheet("Лист1")
sheet2 = client.open_by_url('https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?gid=0#gid=0').worksheet("Лист2")
sheet3 = client.open_by_url('https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?gid=0#gid=0').worksheet("Лист3")

def generate_report(sheet1, sheet2, sheet3):
  sheet1_data = sheet1.get_all_records()
  sheet2_data = sheet2.get_all_records()
  sheet3_data = sheet3.get_all_records()
  students = {row["student_id"]: row for row in sheet1_data}
  payments = {row["student_id"]: row for row in sheet2_data}
  finance = {row["student_id"]: row for row in sheet3_data}

  report_lines = []

    # Дата, на которую проверяем задолженность
  check_date = datetime(2023, 3, 1).date()

    # Интервал между платежами
  interval = timedelta(days=183)

  for student_id, student_info in students.items():

        name = student_info["student_name"]
        has_installment = student_info["installment"] == "Y"

        if student_id not in payments or student_id not in finance:
            continue

        payment_info = payments[student_id]
        finance_info = finance[student_id]

        expected_payment_date = datetime.strptime(
    payment_info["expected_payment_date"], "%d.%m.%Y"
).date()


        # Если к 1 марта 2023 платеж ещё не должен был быть — долгов нет
        if expected_payment_date > check_date:
            continue

        # -------------------------
        # СЧИТАЕМ КОЛИЧЕСТВО ПРОПУЩЕННЫХ ПЛАТЕЖЕЙ
        # -------------------------

        missed_payments = 0
        next_pay_date = expected_payment_date

        while next_pay_date <= check_date:
            missed_payments += 1
            next_pay_date += interval

        # -------------------------
        # ВЫЧИСЛЕНИЕ ДОЛГА
        # -------------------------

        if has_installment:
            # Use 'one-time_payment' as the fixed installment amount for installment plans
            one_payment_amount = finance_info["one-time_payment"]
            left_to_pay = finance_info["left_to_pay"]

            calculated_debt = one_payment_amount * missed_payments
            # Cannot owe more than the total remaining amount
            debt = min(calculated_debt, left_to_pay)

        else:
            # Нет рассрочки → разовый платёж
            one_time_payment_amount = finance_info["one-time_payment"]

            # Если хотя бы один пропуск есть → долг = весь платёж
            debt = one_time_payment_amount if missed_payments > 0 else 0

        # -------------------------
        # Добавляем в отчёт
        # -------------------------
        if debt > 0:
            debt = int(debt)  # округляем сумму
            report_lines.append(f"Студент {name} - долг {debt} рублей")

    # -------------------------
    # СОХРАНЯЕМ В ФАЙЛ
    # -------------------------
  with open("student_debt_report.txt", "w", encoding="utf-8") as file:
        for line in report_lines:
            file.write(line + "\n")
  print("Отчёт student_debt_report.txt успешно создан!")
generate_report(sheet1, sheet2, sheet3)

Отчёт student_debt_report.txt успешно создан!


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [77]:
import pandas as pd

# Здесь будет скачиваться файл с эталонным ответом
# Use -O option to force saving as 'student_debt.txt' to ensure it's read correctly
!wget -O student_debt.txt https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

user_answer = pd.read_csv('student_debt_report.txt', header=None)
correct_answer = pd.read_csv('student_debt.txt', header=None)

--2025-11-18 07:35:19--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.112.3
Connecting to gist.github.com (gist.github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2025-11-18 07:35:19--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt’

student_debt.txt    100%[===================>]  11.06K  --.-KB/s    in 0s      

2025-11-18 07:35:19 (22.4 MB/s)

In [78]:
try:
  # Sort both DataFrames by their single column before comparison
  user_answer_sorted = user_answer.sort_values(by=0).reset_index(drop=True)
  correct_answer_sorted = correct_answer.sort_values(by=0).reset_index(drop=True)

  assert (user_answer_sorted.iloc[:, 0] == correct_answer_sorted.iloc[:, 0]).all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')


Поздравляем, Вы справились и успешно прошли все проверки!!
